# Tech-with-Tim MongoDB tutorial 1

## Connect to MongoDB

In [ ]:
import pprint
from pymongo import MongoClient

client = MongoClient(open("mongo_uri.txt", "r").read().strip())

dbs = client.list_database_names()
test_db = client["test"]
collections = test_db.list_collection_names()
print(collections)

printer = pprint.PrettyPrinter()

### create_documents

In [ ]:
production = client["production"]
person_collection = production["person_collection"]
person_collection.drop()

def create_documents():
    first_names = ["Tim", "Sarah", "Jennifer", "Jose", "Brad", "Allen"]
    last_names = ["Ruscica", "Smith", "Bart", "Cater", "Pit", "Geral"]
    ages = [21, 40, 23, 19, 34, 67]

    docs = []

    for first_name, last_name, age in zip(first_names, last_names, ages):
        doc = {
            "first_name": first_name,
            "last_name": last_name,
            "age": age
        }
        docs.append(doc)
    
    person_collection.insert_many(docs)

create_documents()

### find_all_people

In [ ]:
def find_all_people():
    people = person_collection.find()

    for person in people:
        printer.pprint(person)

find_all_people()

### find_tim

In [ ]:
def find_tim():
    tim = person_collection.find_one({
        "first_name": "Tim",
        "last_name": "Ruscica"
        })
    printer.pprint(tim)

find_tim()

### count_all_people

In [ ]:
def count_all_people():
    count = person_collection.count_documents(filter={})
    print("Number of people: ", count)

count_all_people()

### get_person_by_id

In [ ]:
def get_person_by_id(person_id):
    from bson.objectid import ObjectId
    _id = ObjectId(person_id)
    person = person_collection.find_one({"_id": _id})
    printer.pprint(person)

get_person_by_id("646efc22734bb880162bea47")

### get_age_range

In [ ]:
def get_age_range(min_age, max_age):
    query = {
            "$and": [
                {"age": {"$gte": min_age}},
                {"age": {"$lte": max_age}}
            ]
        }

    people = person_collection.find(query).sort("age")
    for person in people:
        printer.pprint(person)

get_age_range(20, 35)

### project_columns

In [ ]:
def project_columns():
    columns = {"_id": 0, "first_name": 1, "last_name": 1}
    people = person_collection.find({}, columns)
    for person in people:
        printer.pprint(person)

project_columns()

### update_person_by_id

In [ ]:
def update_person_by_id(person_id):
    from bson.objectid import ObjectId

    _id = ObjectId(person_id)

    all_updates = {
        "$set": { "new_field": True },
        "$inc": { "age": 1 },
        "$rename": { "first_name": "first", "last_name": "last" }
    }
    person_collection.update_one({"_id": _id}, all_updates)

update_person_by_id("646efc22734bb880162bea47")

### unset new_field

In [40]:
def __update_person_by_id(person_id):
    from bson.objectid import ObjectId

    _id = ObjectId(person_id)

    person_collection.update_one({"_id": _id}, {"$unset": { "new_field": "" }})

__update_person_by_id("646efc22734bb880162bea47")

### replace_one

In [41]:
def replace_one(person_id):
    from bson.objectid import ObjectId
    _id = ObjectId(person_id)

    new_person = {
        "first_name": "new first name",
        "last_name": "new last name",
        "age": 100,
    }

    person_collection.replace_one({"_id": _id}, new_person)

replace_one("646efc22734bb880162bea47")